### Check GPU availability

In [1]:
!nvidia-smi

Tue Feb  5 15:19:03 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.145                Driver Version: 384.145                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:02:00.0 Off |                  N/A |
| 23%   26C    P8    17W / 250W |      0MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:03:00.0 Off |                  N/A |
| 23%   31C    P8    17W / 250W |      0MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

### Import libraries

In [2]:
import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import copy
import numpy as np
import time
import json
import plotly
import logging
logging.getLogger().setLevel(logging.INFO)

from pprint import pprint
from tqdm import tqdm_notebook
from idst_util import trivial
from idst_util import dstc2

from plotly.graph_objs import Scatter, Layout
from plotly.graph_objs.layout import Margin
plotly.offline.init_notebook_mode(connected = True)

### Check DSTC2 availability and retrieve data

In [3]:
trivial.print_idst()

dstc2.check()

raw_X_train, raw_Y_train, \
raw_X_dev, raw_Y_dev, \
raw_X_test, raw_Y_test, \
ontology = dstc2.retrieve_raw_datasets(train_data_augmentation = True)

INFO:root:+--------------------------------+
INFO:root:|         _ ____  ___________    |
INFO:root:|        (_) __ \/ ___/_  __/    |
INFO:root:|       / / / / /\__ \ / /       |
INFO:root:|      / / /_/ /___/ // /        |
INFO:root:|     /_/_____//____//_/         |
INFO:root:|                                |
INFO:root:+--------------------------------+
INFO:root:|Incremental Dialog State Tracker|
INFO:root:+--------------------------------+
INFO:root:+--------------------------------+
INFO:root:|     Dialog State Tracker 2     |
INFO:root:|         Data Checker           |
INFO:root:+--------------------------------+
INFO:root:Looking for dstc2 directory in .
INFO:root:dstc2 was found!
INFO:root:Looking for dstc2_traindev directory in ./dstc2
INFO:root:dstc2_traindev was found!
INFO:root:Looking for dstc2_test directory in ./dstc2
INFO:root:dstc2_test was found!
INFO:root:Looking for dstc2_scripts directory in ./dstc2
INFO:root:dstc2_scripts was found!
INFO:root:Done!
INFO:root:+-

INFO:root:Extracting raw dev features


INFO:root:Reading dstc2_test.flist
INFO:root:Asserted 1117 dialogs for dstc2_test.flist
INFO:root:Extracting raw test features


INFO:root:Done!


### Set device

In [4]:
logging.info("+--------------------------------+")
logging.info("|             Device             |")
logging.info("+--------------------------------+")

GPU_ID = 1
DEVICE = torch.device("cuda:{}".format(GPU_ID) if torch.cuda.is_available() else "cpu")
if str(DEVICE) == "cpu":
    logging.warning("Running on CPU")
else:
    logging.info("Running on GPU {}".format(GPU_ID))

INFO:root:+--------------------------------+
INFO:root:|             Device             |
INFO:root:+--------------------------------+
INFO:root:Running on GPU 1


### Create vocabularies

In [5]:
logging.info("+--------------------------------+")
logging.info("|          Vocabulary            |")
logging.info("+--------------------------------+")
logging.info("Creating token_to_index, index_to_token and token_to_count dictionaries")

token_to_index = {"<unk>": 0}
index_to_token = {0: "<unk>"}
token_to_count = {"<unk>": 1}

for raw_train_dialog in tqdm_notebook(raw_X_train):
    for raw_train_turn in raw_train_dialog["turns"]:
        tokens_scores = raw_train_turn["system"] + raw_train_turn["user"]
        for token_score in tokens_scores:
            token = token_score[0]
            if token not in token_to_index:
                token_to_index[token] = len(token_to_index)
                index_to_token[len(token_to_index)] = token
                token_to_count[token] = 1
            else:
                token_to_count[token] += 1
                
assert len(token_to_index) == len(index_to_token)
assert len(token_to_index) == len(token_to_count)

logging.info("Done!")

INFO:root:+--------------------------------+
INFO:root:|          Vocabulary            |
INFO:root:+--------------------------------+
INFO:root:Creating token_to_index, index_to_token and token_to_count dictionaries


INFO:root:Done!


### Execution configuration

In [6]:
logging.info("+--------------------------------+")
logging.info("|         Configuration          |")
logging.info("+--------------------------------+")

VOCABULARY_SIZE = len(token_to_index)

# NOTE: we add +2 because of null and dontcare cases
GOAL_FOOD_DIM = len(ontology["informable"]["food"]) + 2 
GOAL_PRICERANGE_DIM = len(ontology["informable"]["pricerange"]) + 2
GOAL_NAME_DIM = len(ontology["informable"]["name"]) + 2
GOAL_AREA_DIM = len(ontology["informable"]["area"]) + 2

METHOD_DIM = len(ontology["method"])

REQUESTED_DIM = len(ontology["requestable"])

EMBEDDING_DIM = 170
ALTERED_EMBEDDING_DIM = 300
HIDDEN_DIM = 100

NUM_EPOCHS = 50
BATCH_SIZE = 10

GOAL_LOSS_FUNCTION = nn.CrossEntropyLoss()
METHOD_LOSS_FUNCTION = nn.CrossEntropyLoss()
REQUESTED_LOSS_FUNCTION = nn.BCELoss()

logging.info("VOCABULARY_SIZE:\t\t\t{}".format(VOCABULARY_SIZE))

logging.info("GOAL_FOOD_DIM:\t\t\t{}".format(GOAL_FOOD_DIM))
logging.info("GOAL_PRICERANGE_DIM:\t\t\t{}".format(GOAL_PRICERANGE_DIM))
logging.info("GOAL_NAME_DIM:\t\t\t{}".format(GOAL_NAME_DIM))
logging.info("GOAL_AREA_DIM:\t\t\t{}".format(GOAL_AREA_DIM))

logging.info("METHOD_DIM:\t\t\t\t{}".format(METHOD_DIM))

logging.info("REQUESTED_DIM:\t\t\t{}".format(REQUESTED_DIM))

logging.info("EMBEDDING_DIM:\t\t\t{}".format(EMBEDDING_DIM))
logging.info("ALTERED_EMBEDDING_DIM:\t\t{}".format(ALTERED_EMBEDDING_DIM))
logging.info("HIDDEN_DIM:\t\t\t\t{}".format(HIDDEN_DIM))

logging.info("NUM_EPOCHS:\t\t\t\t{}".format(NUM_EPOCHS))
logging.info("BATCH_SIZE:\t\t\t\t{}".format(BATCH_SIZE))

logging.info("GOAL_LOSS_FUNCTION:\t\t\t{}".format(GOAL_LOSS_FUNCTION))
logging.info("METHOD_LOSS_FUNCTION:\t\t\t{}".format(METHOD_LOSS_FUNCTION))
logging.info("REQUESTED_LOSS_FUNCTION:\t\t{}".format(REQUESTED_LOSS_FUNCTION))

INFO:root:+--------------------------------+
INFO:root:|         Configuration          |
INFO:root:+--------------------------------+
INFO:root:VOCABULARY_SIZE:			1137
INFO:root:GOAL_FOOD_DIM:			93
INFO:root:GOAL_PRICERANGE_DIM:			5
INFO:root:GOAL_NAME_DIM:			115
INFO:root:GOAL_AREA_DIM:			7
INFO:root:METHOD_DIM:				5
INFO:root:REQUESTED_DIM:			8
INFO:root:EMBEDDING_DIM:			170
INFO:root:ALTERED_EMBEDDING_DIM:		300
INFO:root:HIDDEN_DIM:				100
INFO:root:NUM_EPOCHS:				50
INFO:root:BATCH_SIZE:				10
INFO:root:GOAL_LOSS_FUNCTION:			CrossEntropyLoss()
INFO:root:METHOD_LOSS_FUNCTION:			CrossEntropyLoss()
INFO:root:REQUESTED_LOSS_FUNCTION:		BCELoss()


### Utilities

In [31]:
def get_index_and_score(turn, token_to_index, mode, device):
    
    indices = []
    scores = []
    
    if mode == "train": # LecTrack 4.3: Out-of-Vocabulary Words
        for system_token, system_token_score in turn["system"]:
            indices.append(token_to_index[system_token])
            scores.append(system_token_score)
        for user_token, user_token_score in turn["user"]:
            if np.random.binomial(n = 1, p = 0.1) == 1:
                indices.append(token_to_index["<unk>"])
            else:
                indices.append(token_to_index[user_token])
            scores.append(user_token_score)
    else:
        tokens_scores = turn["system"] + turn["user"]
        for token, score in tokens_scores:
            if token not in token_to_index:
                indices.append(token_to_index["<unk>"])
            else:
                indices.append(token_to_index[token])
            scores.append(score)
            
    assert len(indices) == len(scores)
    
    return torch.tensor(indices, dtype = torch.long, device = device), torch.tensor(scores, dtype = torch.float, device = device)

# --------------------

class EarlyStopping():
    
    def __init__(self, min_delta = 0, patience = 0):
        
        self.min_delta = min_delta
        self.patience = patience
        self.wait = 0
        self.stopped_epoch = 0
        self.best = -np.Inf
        self.stop_training = False
    
    def on_epoch_end(self, epoch, current_value):
        if np.greater((current_value - self.min_delta), self.best):
            self.best = current_value
            self.wait = 0
        else:
            self.wait += 1
            if self.wait > self.patience:
                self.stopped_epoch = epoch
                self.stop_training = True
        return self.stop_training

# --------------------

def make_tracker(model_GoalPricerange, model_GoalArea, model_GoalName, model_GoalFood, model_Requested, model_Method, raw_X, raw_Y, dataset, percentage = 1.0):
    
    model_GoalPricerange = model_GoalPricerange.eval()
    model_GoalArea = model_GoalArea.eval()
    model_GoalName = model_GoalName.eval()
    model_GoalFood = model_GoalFood.eval()
    model_Requested = model_Requested.eval()
    model_Method = model_Method.eval()
    
    with torch.no_grad():
        tracker_json = {}
        tracker_json["dataset"] = dataset
        tracker_json["sessions"] = []

        start_time = time.time()
        
        for raw_X_dialog, raw_Y_dialog in tqdm_notebook(zip(raw_X, raw_Y), total = len(raw_X)):
            
            model_GoalPricerange.hidden = model_GoalPricerange.init_hidden()
            model_GoalArea.hidden = model_GoalArea.init_hidden()
            model_GoalName.hidden = model_GoalName.init_hidden()
            model_GoalFood.hidden = model_GoalFood.init_hidden()
            model_Requested.hidden = model_Requested.init_hidden()
            model_Method.hidden = model_Method.init_hidden()
            
            session = {}
            session["session-id"] = raw_X_dialog["session-id"]
            session["turns"] = []

            for raw_X_turn, raw_Y_turn in zip(raw_X_dialog["turns"], raw_Y_dialog["turns"]):
                
                turn = {}
                turn["goal-labels"] = {}

                indices, scores = get_index_and_score(raw_X_turn, token_to_index, mode = "eval", device = DEVICE)
                
                incremental_index = int(np.around(percentage * len(indices))) - 1
                
                goal_foods = model_GoalFood(indices, scores)
                goal_food = goal_foods[incremental_index]
                
                goal_priceranges = model_GoalPricerange(indices, scores)
                goal_pricerange = goal_priceranges[incremental_index]
                
                goal_names = model_GoalName(indices, scores)
                goal_name = goal_names[incremental_index]
                
                goal_areas = model_GoalArea(indices, scores)
                goal_area = goal_areas[incremental_index]
                
                requesteds = model_Requested(indices, scores)
                requested = requesteds[incremental_index]
                
                methods = model_Method(indices, scores)
                method = methods[incremental_index]
                
                turn["goal-labels"]["food"] = retrieve_output_GoalFood(goal_food, ontology)
                turn["goal-labels"]["pricerange"] = retrieve_output_GoalPricerange(goal_pricerange, ontology)
                turn["goal-labels"]["name"] = retrieve_output_GoalName(goal_name, ontology)
                turn["goal-labels"]["area"] = retrieve_output_GoalArea(goal_area, ontology)
                turn["requested-slots"] = retrieve_output_Requested(requested, ontology)
                turn["method-label"] = retrieve_output_Method(method, ontology)
                
                session["turns"].append(turn)
                
            tracker_json["sessions"].append(session)
            
        end_time = time.time()
        tracker_json["wall-time"] = end_time - start_time
        
        return tracker_json

# --------------------
    
def get_scores(tracker, dataset, predictor = False):
    
    goal_pricerange_accuracy = None
    goal_pricerange_l2 = None
    
    goal_area_accuracy = None
    goal_area_l2 = None
    
    goal_name_accuracy = None
    goal_name_l2 = None
    
    goal_food_accuracy = None
    goal_food_l2 = None
    
    goal_accuracy = None
    goal_l2 = None
    
    requested_accuracy = None
    requested_l2 = None
    
    method_accuracy = None
    method_l2 = None
    
    with open("tracker_panfmr_incremental.json", "w") as tracker_file:
        json.dump(tracker, tracker_file)
    
    if predictor:
        
        if dataset == "dstc2_train":
            !python3 dstc2/dstc2_scripts/score_predictor.py\
            --dataset dstc2_train\
            --dataroot dstc2/dstc2_traindev/data\
            --ontology dstc2/dstc2_scripts/config/ontology_dstc2.json\
            --trackfile tracker_panfmr_incremental.json\
            --scorefile tracker_panfmr_incremental.score.csv
        elif dataset == "dstc2_dev":
            !python3 dstc2/dstc2_scripts/score_predictor.py\
            --dataset dstc2_dev\
            --dataroot dstc2/dstc2_traindev/data\
            --ontology dstc2/dstc2_scripts/config/ontology_dstc2.json\
            --trackfile tracker_panfmr_incremental.json\
            --scorefile tracker_panfmr_incremental.score.csv
        else:
            !python3 dstc2/dstc2_scripts/score_predictor.py\
            --dataset dstc2_test\
            --dataroot dstc2/dstc2_test/data\
            --ontology dstc2/dstc2_scripts/config/ontology_dstc2.json\
            --trackfile tracker_panfmr_incremental.json\
            --scorefile tracker_panfmr_incremental.score.csv
            
    else: # predictor == False
        
        if dataset == "dstc2_train":
            !python3 dstc2/dstc2_scripts/score.py\
            --dataset dstc2_train\
            --dataroot dstc2/dstc2_traindev/data\
            --ontology dstc2/dstc2_scripts/config/ontology_dstc2.json\
            --trackfile tracker_panfmr_incremental.json\
            --scorefile tracker_panfmr_incremental.score.csv
        elif dataset == "dstc2_dev":
            !python3 dstc2/dstc2_scripts/score.py\
            --dataset dstc2_dev\
            --dataroot dstc2/dstc2_traindev/data\
            --ontology dstc2/dstc2_scripts/config/ontology_dstc2.json\
            --trackfile tracker_panfmr_incremental.json\
            --scorefile tracker_panfmr_incremental.score.csv
        else:
            !python3 dstc2/dstc2_scripts/score.py\
            --dataset dstc2_test\
            --dataroot dstc2/dstc2_test/data\
            --ontology dstc2/dstc2_scripts/config/ontology_dstc2.json\
            --trackfile tracker_panfmr_incremental.json\
            --scorefile tracker_panfmr_incremental.score.csv
        
    score_file_cat = !cat tracker_panfmr_incremental.score.csv
    
    for line in score_file_cat:
        if line.startswith("goal.pricerange, acc, 2, a"):
            value = line.split(",")[-1]
            if "-" not in value:
                goal_pricerange_accuracy = float(value)
        if line.startswith("goal.pricerange, l2, 2, a"):
            value = line.split(",")[-1]
            if "-" not in value:
                goal_pricerange_l2 = float(value)
        if line.startswith("goal.area, acc, 2, a"):
            value = line.split(",")[-1]
            if "-" not in value:
                goal_area_accuracy = float(value)
        if line.startswith("goal.area, l2, 2, a"):
            value = line.split(",")[-1]
            if "-" not in value:
                goal_area_l2 = float(value)
        if line.startswith("goal.name, acc, 2, a"):
            value = line.split(",")[-1]
            if "-" not in value:
                goal_name_accuracy = float(value)
        if line.startswith("goal.name, l2, 2, a"):
            value = line.split(",")[-1]
            if "-" not in value:
                goal_name_l2 = float(value)
        if line.startswith("goal.food, acc, 2, a"):
            value = line.split(",")[-1]
            if "-" not in value:
                goal_food_accuracy = float(value)
        if line.startswith("goal.food, l2, 2, a"):
            value = line.split(",")[-1]
            if "-" not in value:
                goal_food_l2 = float(value)
        if line.startswith("goal.joint, acc, 2, a"):
            value = line.split(",")[-1]
            if "-" not in value:
                goal_accuracy = float(value)
        if line.startswith("goal.joint, l2, 2, a"):
            value = line.split(",")[-1]
            if "-" not in value:
                goal_l2 = float(value)
        if line.startswith("requested.all, acc, 2, a"):
            value = line.split(",")[-1]
            if "-" not in value:
                requested_accuracy = float(value)
        if line.startswith("requested.all, l2, 2, a"):
            value = line.split(",")[-1]
            if "-" not in value:
                requested_l2 = float(value)
        if line.startswith("method, acc, 2, a"):
            value = line.split(",")[-1]
            if "-" not in value:
                method_accuracy = float(value)
        if line.startswith("method, l2, 2, a"):
            value = line.split(",")[-1]
            if "-" not in value:
                method_l2 = float(value)
    
    scores_dict = {}
    scores_dict["goal_pricerange_accuracy"] = goal_pricerange_accuracy
    scores_dict["goal_pricerange_l2"] = goal_pricerange_l2
    scores_dict["goal_area_accuracy"] = goal_area_accuracy
    scores_dict["goal_area_l2"] = goal_area_l2
    scores_dict["goal_name_accuracy"] = goal_name_accuracy
    scores_dict["goal_name_l2"] = goal_name_l2
    scores_dict["goal_food_accuracy"] = goal_food_accuracy
    scores_dict["goal_food_l2"] = goal_food_l2
    scores_dict["goal_accuracy"] = goal_accuracy
    scores_dict["goal_l2"] = goal_l2
    scores_dict["requested_accuracy"] = requested_accuracy
    scores_dict["requested_l2"] = requested_l2
    scores_dict["method_accuracy"] = method_accuracy
    scores_dict["method_l2"] = method_l2
    
    return scores_dict

# --------------------

def retrieve_gold_GoalPricerange(raw_Y, ontology, device):
    ontology_informable_pricerange = ontology["informable"]["pricerange"]
    raw_goal_pricerange = raw_Y["goal"]["pricerange"]
    goal_pricerange = 0
    if raw_goal_pricerange != None:
        if raw_goal_pricerange == "dontcare":
            goal_pricerange = 1
        else:    
            goal_pricerange = ontology_informable_pricerange.index(raw_goal_pricerange) + 2
    return torch.tensor([goal_pricerange], dtype = torch.long, device = device)

def retrieve_output_GoalPricerange(output_tensor, ontology):
    ontology_informable_pricerange = ontology["informable"]["pricerange"]
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    goal_pricerange_dict = {}
    goal_pricerange_dict["dontcare"] = output_tensor[1].item()
    for index in range(len(output_tensor) - 2):     
        goal_pricerange_dict[ontology_informable_pricerange[index]] = output_tensor[index + 2].item()
    return goal_pricerange_dict

def retrieve_gold_GoalArea(raw_Y, ontology, device):
    ontology_informable_area = ontology["informable"]["area"]
    raw_goal_area = raw_Y["goal"]["area"]
    goal_area = 0
    if raw_goal_area != None:
        if raw_goal_area == "dontcare":
            goal_area = 1
        else:    
            goal_area = ontology_informable_area.index(raw_goal_area) + 2
    return torch.tensor([goal_area], dtype = torch.long, device = device)

def retrieve_output_GoalArea(output_tensor, ontology):
    ontology_informable_area = ontology["informable"]["area"]
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    goal_area_dict = {}
    goal_area_dict["dontcare"] = output_tensor[1].item()
    for index in range(len(output_tensor) - 2):
        goal_area_dict[ontology_informable_area[index]] = output_tensor[index + 2].item()
    return goal_area_dict

def retrieve_gold_GoalName(raw_Y, ontology, device):
    ontology_informable_name = ontology["informable"]["name"]
    raw_goal_name = raw_Y["goal"]["name"]
    goal_name = 0
    if raw_goal_name != None:
        if raw_goal_name == "dontcare":
            goal_name = 1
        else:    
            goal_name = ontology_informable_name.index(raw_goal_name) + 2
    return torch.tensor([goal_name], dtype = torch.long, device = device)

def retrieve_output_GoalName(output_tensor, ontology):
    ontology_informable_name = ontology["informable"]["name"]
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    goal_name_dict = {}
    goal_name_dict["dontcare"] = output_tensor[1].item()
    for index in range(len(output_tensor) - 2):
        goal_name_dict[ontology_informable_name[index]] = output_tensor[index + 2].item()
    return goal_name_dict

def retrieve_gold_GoalFood(raw_Y, ontology, device):
    ontology_informable_food = ontology["informable"]["food"]
    raw_goal_food = raw_Y["goal"]["food"]
    goal_food = 0
    if raw_goal_food != None:
        if raw_goal_food == "dontcare":
            goal_food = 1
        else:    
            goal_food = ontology_informable_food.index(raw_goal_food) + 2
    return torch.tensor([goal_food], dtype = torch.long, device = device)

def retrieve_output_GoalFood(output_tensor, ontology):
    ontology_informable_food = ontology["informable"]["food"]
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    goal_food_dict = {}
    goal_food_dict["dontcare"] = output_tensor[1].item() 
    for index in range(len(output_tensor) - 2):
        goal_food_dict[ontology_informable_food[index]] = output_tensor[index + 2].item()
    return goal_food_dict

def retrieve_gold_Requested(raw_Y, ontology, device):
    ontology_requestable = ontology["requestable"]
    raw_gold_requested = raw_Y["requested"]
    gold_requested = np.zeros(len(ontology_requestable), dtype = float)
    if len(raw_gold_requested) != 0:
        for requested in raw_gold_requested:
            gold_requested[ontology_requestable.index(requested)] = 1.0
    return torch.tensor([gold_requested], dtype = torch.float, device = device)

def retrieve_output_Requested(output_tensor, ontology):
    ontology_requestable = ontology["requestable"]
    output_tensor = output_tensor.view(-1)
    requested_dict = {}
    for index in range(len(output_tensor)):
        probability_value = output_tensor[index].item()
        requested_dict[ontology_requestable[index]] = probability_value
    return requested_dict

def retrieve_gold_Method(raw_Y, ontology, device):
    ontology_method = ontology["method"]
    raw_gold_method = raw_Y["method"]
    gold_method = ontology_method.index(raw_gold_method)
    return torch.tensor([gold_method], dtype = torch.long, device = device)

def retrieve_output_Method(output_tensor, ontology):
    ontology_method = ontology["method"]
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    method_dict = {}
    for index in range(len(output_tensor)):
        method_dict[ontology_method[index]] = output_tensor[index].item()
    return method_dict

### iDST Model

In [8]:
class iDSTModel(nn.Module):
    
    def __init__(self, vocabulary_size, embedding_dim, altered_embedding_dim, hidden_dim, output_dim, device, is_requested = False):
        super(iDSTModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.is_requested = is_requested
        self.device = device
        self.output_dim = output_dim
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size, embedding_dim = embedding_dim)
        self.altered_embeddings = nn.Linear(in_features = (embedding_dim + 1), out_features = altered_embedding_dim) # +1 for the ASR-score
        self.lstm = nn.LSTM(input_size = altered_embedding_dim, hidden_size = hidden_dim)
        self.classifier = nn.Linear(in_features = hidden_dim, out_features = output_dim)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.zeros(1, 1, self.hidden_dim, device = self.device),
                torch.zeros(1, 1, self.hidden_dim, device = self.device))

    def forward(self, indices, scores):
        embeddings = self.embeddings(indices)
        embeddings_concat_score = torch.cat((embeddings, scores.unsqueeze(dim = 1)), dim = 1) 
        altered_embeddings = F.relu(self.altered_embeddings(embeddings_concat_score))
        lstm_out, self.hidden = self.lstm(altered_embeddings.view(len(indices), 1, -1), self.hidden)
        if self.is_requested:
            output_tensor = torch.sigmoid(self.classifier(lstm_out).view(-1, self.output_dim))
        else:
            output_tensor = F.log_softmax(self.classifier(lstm_out).view(-1, self.output_dim), dim = 1)
        return output_tensor

### Goal Pricerange Model

In [9]:
model_GoalPricerange = iDSTModel(vocabulary_size = VOCABULARY_SIZE,
                                 embedding_dim = EMBEDDING_DIM,
                                 altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                 hidden_dim = HIDDEN_DIM,
                                 output_dim = GOAL_PRICERANGE_DIM,
                                 device = DEVICE).to(DEVICE)
optimizer_GoalPricerange = optim.Adam(model_GoalPricerange.parameters(), lr = 1e-3, amsgrad = True) 

### Goal Area Model

In [10]:
model_GoalArea = iDSTModel(vocabulary_size = VOCABULARY_SIZE,
                           embedding_dim = EMBEDDING_DIM,
                           altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                           hidden_dim = HIDDEN_DIM,
                           output_dim = GOAL_AREA_DIM,
                           device = DEVICE).to(DEVICE)
optimizer_GoalArea = optim.Adam(model_GoalArea.parameters(), lr = 1e-3, amsgrad = True) 

### Goal Name Model

In [11]:
model_GoalName = iDSTModel(vocabulary_size = VOCABULARY_SIZE,
                           embedding_dim = EMBEDDING_DIM,
                           altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                           hidden_dim = HIDDEN_DIM,
                           output_dim = GOAL_NAME_DIM,
                           device = DEVICE).to(DEVICE)
optimizer_GoalName = optim.Adam(model_GoalName.parameters(), lr = 1e-3, amsgrad = True) 

### Goal Food Model

In [12]:
model_GoalFood = iDSTModel(vocabulary_size = VOCABULARY_SIZE,
                           embedding_dim = EMBEDDING_DIM,
                           altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                           hidden_dim = HIDDEN_DIM,
                           output_dim = GOAL_FOOD_DIM,
                           device = DEVICE).to(DEVICE)
optimizer_GoalFood = optim.Adam(model_GoalFood.parameters(), lr = 1e-3, amsgrad = True) 

### Requested Model

In [13]:
model_Requested = iDSTModel(vocabulary_size = VOCABULARY_SIZE,
                            embedding_dim = EMBEDDING_DIM,
                            altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                            hidden_dim = HIDDEN_DIM,
                            output_dim = REQUESTED_DIM,
                            device = DEVICE,
                            is_requested = True).to(DEVICE)
optimizer_Requested = optim.Adam(model_Requested.parameters(), lr = 1e-3, amsgrad = True) 

### Method Model

In [14]:
model_Method = iDSTModel(vocabulary_size = VOCABULARY_SIZE,
                         embedding_dim = EMBEDDING_DIM,
                         altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                         hidden_dim = HIDDEN_DIM,
                         output_dim = METHOD_DIM,
                         device = DEVICE).to(DEVICE)
optimizer_Method = optim.Adam(model_Method.parameters(), lr = 1e-3, amsgrad = True)

### Train iDST

In [15]:
goal_pricerange_early_stopping = EarlyStopping(patience = 4)
goal_area_early_stopping = EarlyStopping(patience = 4)
goal_name_early_stopping = EarlyStopping(patience = 4)
goal_food_early_stopping = EarlyStopping(patience = 4)
requested_early_stopping = EarlyStopping(patience = 4)
method_early_stopping = EarlyStopping(patience = 4)

train_indices_loader = torch.utils.data.DataLoader(np.arange(raw_X_train.shape[0]), batch_size = BATCH_SIZE, shuffle = True)

for epoch in range(NUM_EPOCHS):
    
    logging.info("Epoch\t{}/{}".format(epoch + 1, NUM_EPOCHS))
    
    if not goal_pricerange_early_stopping.stop_training:
        model_GoalPricerange = model_GoalPricerange.train()
    if not goal_area_early_stopping.stop_training:
        model_GoalArea = model_GoalArea.train()
    if not goal_name_early_stopping.stop_training:
        model_GoalName = model_GoalName.train()
    if not goal_food_early_stopping.stop_training:
        model_GoalFood = model_GoalFood.train()
    if not requested_early_stopping.stop_training:
        model_Requested = model_Requested.train()
    if not method_early_stopping.stop_training:
        model_Method = model_Method.train()
    
    for train_indices in tqdm_notebook(train_indices_loader, total = len(train_indices_loader)):
        
        if not goal_pricerange_early_stopping.stop_training:
            optimizer_GoalPricerange.zero_grad()
            goal_pricerange_accumulated_loss = 0
        if not goal_area_early_stopping.stop_training:
            optimizer_GoalArea.zero_grad()
            goal_area_accumulated_loss = 0
        if not goal_name_early_stopping.stop_training:
            optimizer_GoalName.zero_grad()
            goal_name_accumulated_loss = 0
        if not goal_food_early_stopping.stop_training:
            optimizer_GoalFood.zero_grad()
            goal_food_accumulated_loss = 0
        if not requested_early_stopping.stop_training:
            optimizer_Requested.zero_grad()
            requested_accumulated_loss = 0
        if not method_early_stopping.stop_training:
            optimizer_Method.zero_grad()
            method_accumulated_loss = 0
        
        for raw_X_train_dialog, raw_Y_train_dialog in zip(raw_X_train[train_indices], raw_Y_train[train_indices]):
            
            if not goal_pricerange_early_stopping.stop_training:
                model_GoalPricerange.hidden = model_GoalPricerange.init_hidden()
            if not goal_area_early_stopping.stop_training:
                model_GoalArea.hidden = model_GoalArea.init_hidden()
            if not goal_name_early_stopping.stop_training:
                model_GoalName.hidden = model_GoalName.init_hidden()
            if not goal_food_early_stopping.stop_training:
                model_GoalFood.hidden = model_GoalFood.init_hidden()
            if not requested_early_stopping.stop_training:
                model_Requested.hidden = model_Requested.init_hidden()
            if not method_early_stopping.stop_training:
                model_Method.hidden = model_Method.init_hidden()
                
            for raw_X_train_turn, raw_Y_train_turn in zip(raw_X_train_dialog["turns"], raw_Y_train_dialog["turns"]):

                indices, scores = get_index_and_score(raw_X_train_turn, token_to_index, mode = "train", device = DEVICE)
                
                if not goal_pricerange_early_stopping.stop_training:
                    goal_pricerange_outputs = model_GoalPricerange(indices, scores)
                    for goal_pricerange in goal_pricerange_outputs:
                        goal_pricerange = goal_pricerange.unsqueeze(0)
                        goal_pricerange_loss = GOAL_LOSS_FUNCTION(goal_pricerange, retrieve_gold_GoalPricerange(raw_Y_train_turn, ontology = ontology, device = DEVICE))
                        goal_pricerange_accumulated_loss += goal_pricerange_loss
                if not goal_area_early_stopping.stop_training:
                    goal_area_outputs = model_GoalArea(indices, scores)
                    for goal_area in goal_area_outputs:
                        goal_area = goal_area.unsqueeze(0)
                        goal_area_loss = GOAL_LOSS_FUNCTION(goal_area, retrieve_gold_GoalArea(raw_Y_train_turn, ontology = ontology, device = DEVICE))
                        goal_area_accumulated_loss += goal_area_loss
                if not goal_name_early_stopping.stop_training:
                    goal_name_outputs = model_GoalName(indices, scores)
                    for goal_name in goal_name_outputs:
                        goal_name = goal_name.unsqueeze(0)
                        goal_name_loss = GOAL_LOSS_FUNCTION(goal_name, retrieve_gold_GoalName(raw_Y_train_turn, ontology = ontology, device = DEVICE))
                        goal_name_accumulated_loss += goal_name_loss
                if not goal_food_early_stopping.stop_training:
                    goal_food_outputs = model_GoalFood(indices, scores)
                    for goal_food in goal_food_outputs:
                        goal_food = goal_food.unsqueeze(0)
                        goal_food_loss = GOAL_LOSS_FUNCTION(goal_food, retrieve_gold_GoalFood(raw_Y_train_turn, ontology = ontology, device = DEVICE))
                        goal_food_accumulated_loss += goal_food_loss
                if not requested_early_stopping.stop_training:
                    requested_outputs = model_Requested(indices, scores)
                    for requested in requested_outputs:
                        requested = requested.unsqueeze(0)
                        requested_loss = REQUESTED_LOSS_FUNCTION(requested, retrieve_gold_Requested(raw_Y_train_turn, ontology = ontology, device = DEVICE))
                        requested_accumulated_loss += requested_loss
                if not method_early_stopping.stop_training:
                    method_outputs = model_Method(indices, scores)
                    for method in method_outputs:
                        method = method.unsqueeze(0)
                        method_loss = METHOD_LOSS_FUNCTION(method, retrieve_gold_Method(raw_Y_train_turn, ontology = ontology, device = DEVICE))
                        method_accumulated_loss += method_loss
        
        if not goal_pricerange_early_stopping.stop_training:
            goal_pricerange_accumulated_loss.backward()
            optimizer_GoalPricerange.step()
        if not goal_area_early_stopping.stop_training:
            goal_area_accumulated_loss.backward()
            optimizer_GoalArea.step()
        if not goal_name_early_stopping.stop_training:
            goal_name_accumulated_loss.backward()
            optimizer_GoalName.step()
        if not goal_food_early_stopping.stop_training:
            goal_food_accumulated_loss.backward()
            optimizer_GoalFood.step()
        if not requested_early_stopping.stop_training:
            requested_accumulated_loss.backward()
            optimizer_Requested.step()
        if not method_early_stopping.stop_training:
            method_accumulated_loss.backward()
            optimizer_Method.step()
        
    dev_tracker = make_tracker(model_GoalPricerange, model_GoalArea, model_GoalName, model_GoalFood, model_Requested, model_Method,
                               raw_X_dev, raw_Y_dev, dataset = "dstc2_dev", percentage = 1.0)
    
    dev_scores_dict = get_scores(dev_tracker, dataset = "dstc2_dev")
    
    logging.info("DEV Goal Pricerange Acc: {}".format(dev_scores_dict["goal_pricerange_accuracy"]))
    logging.info("DEV Goal Area Acc: {}".format(dev_scores_dict["goal_area_accuracy"]))
    logging.info("DEV Goal Name Acc: {}".format(dev_scores_dict["goal_name_accuracy"]))
    logging.info("DEV Goal Food Acc: {}".format(dev_scores_dict["goal_food_accuracy"]))
    
    logging.info("DEV Acc:\t\t{}({})\t\t{}({})\t\t{}({})".format(dev_scores_dict["goal_accuracy"],
                                                                 np.around(dev_scores_dict["goal_accuracy"], decimals = 2),
                                                                 dev_scores_dict["requested_accuracy"],
                                                                 np.around(dev_scores_dict["requested_accuracy"], decimals = 2),
                                                                 dev_scores_dict["method_accuracy"],
                                                                 np.around(dev_scores_dict["method_accuracy"], decimals = 2)))
    
    logging.info("DEV L2:\t\t{}({})\t\t{}({})\t\t{}({})".format(dev_scores_dict["goal_l2"],
                                                                np.around(dev_scores_dict["goal_l2"], decimals = 2),
                                                                dev_scores_dict["requested_l2"],
                                                                np.around(dev_scores_dict["requested_l2"], decimals = 2),
                                                                dev_scores_dict["method_l2"],
                                                                np.around(dev_scores_dict["method_l2"], decimals = 2)))

    goal_pricerange_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = (dev_scores_dict["goal_pricerange_accuracy"]))
    goal_area_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = (dev_scores_dict["goal_area_accuracy"]))
    goal_name_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = (dev_scores_dict["goal_name_accuracy"]))
    goal_food_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = (dev_scores_dict["goal_food_accuracy"]))
    requested_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = (dev_scores_dict["requested_accuracy"]))
    method_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = (dev_scores_dict["method_accuracy"]))
    
    if goal_pricerange_early_stopping.wait == 0:
        torch.save(model_GoalPricerange.state_dict(), "model_PANFMR_incremental_GoalPricerange.pt")
    if goal_area_early_stopping.wait == 0:
        torch.save(model_GoalArea.state_dict(), "model_PANFMR_incremental_GoalArea.pt")
    if goal_name_early_stopping.wait == 0:
        torch.save(model_GoalName.state_dict(), "model_PANFMR_incremental_GoalName.pt")
    if goal_food_early_stopping.wait == 0:
        torch.save(model_GoalFood.state_dict(), "model_PANFMR_incremental_GoalFood.pt")
    if requested_early_stopping.wait == 0:
        torch.save(model_Requested.state_dict(), "model_PANFMR_incremental_Requested.pt")
    if method_early_stopping.wait == 0:
        torch.save(model_Method.state_dict(), "model_PANFMR_incremental_Method.pt")
        
    if goal_pricerange_early_stopping.stop_training:
        model_GoalPricerange.load_state_dict(torch.load("model_PANFMR_incremental_GoalPricerange.pt"))
    if goal_area_early_stopping.stop_training:
        model_GoalArea.load_state_dict(torch.load("model_PANFMR_incremental_GoalArea.pt"))
    if goal_name_early_stopping.stop_training:
        model_GoalName.load_state_dict(torch.load("model_PANFMR_incremental_GoalName.pt"))
    if goal_food_early_stopping.stop_training:
        model_GoalFood.load_state_dict(torch.load("model_PANFMR_incremental_GoalFood.pt"))
    if requested_early_stopping.stop_training:
        model_Requested.load_state_dict(torch.load("model_PANFMR_incremental_Requested.pt"))
    if method_early_stopping.stop_training:
        model_Method.load_state_dict(torch.load("model_PANFMR_incremental_Method.pt"))
        
    if goal_pricerange_early_stopping.stop_training and goal_area_early_stopping.stop_training and goal_name_early_stopping.stop_training and \
        goal_food_early_stopping.stop_training and requested_early_stopping.stop_training and method_early_stopping.stop_training:
        break

INFO:root:Epoch	1/50


INFO:root:DEV Goal Pricerange Acc: 0.8891105
INFO:root:DEV Goal Area Acc: 0.8484576
INFO:root:DEV Goal Name Acc: 0.9065421
INFO:root:DEV Goal Food Acc: 0.4007989
INFO:root:DEV Acc:		0.3860792(0.39)		0.8597187(0.86)		0.8864459(0.89)
INFO:root:DEV L2:		0.7833443(0.78)		0.2222096(0.22)		0.1859387(0.19)
INFO:root:Epoch	2/50


INFO:root:DEV Goal Pricerange Acc: 0.9186278
INFO:root:DEV Goal Area Acc: 0.8658281
INFO:root:DEV Goal Name Acc: 0.9065421
INFO:root:DEV Goal Food Acc: 0.6784288
INFO:root:DEV Acc:		0.5750782(0.58)		0.9134511(0.91)		0.8898086(0.89)
INFO:root:DEV L2:		0.6030542(0.6)		0.1386777(0.14)		0.1810151(0.18)
INFO:root:Epoch	3/50


INFO:root:DEV Goal Pricerange Acc: 0.9070602
INFO:root:DEV Goal Area Acc: 0.8718179
INFO:root:DEV Goal Name Acc: 0.9065421
INFO:root:DEV Goal Food Acc: 0.698735
INFO:root:DEV Acc:		0.5982795(0.6)		0.9444645(0.94)		0.8934299(0.89)
INFO:root:DEV L2:		0.5538399(0.55)		0.097616(0.1)		0.1813849(0.18)
INFO:root:Epoch	4/50


INFO:root:DEV Goal Pricerange Acc: 0.8911049
INFO:root:DEV Goal Area Acc: 0.8604373
INFO:root:DEV Goal Name Acc: 0.9065421
INFO:root:DEV Goal Food Acc: 0.7150466
INFO:root:DEV Acc:		0.5870699(0.59)		0.9433826(0.94)		0.88179(0.88)
INFO:root:DEV L2:		0.5752234(0.58)		0.0906178(0.09)		0.1977969(0.2)
INFO:root:Epoch	5/50


...

INFO:root:DEV Goal Pricerange Acc: 0.9313921
INFO:root:DEV Goal Area Acc: 0.8751123
INFO:root:DEV Goal Name Acc: 0.9065421
INFO:root:DEV Goal Food Acc: 0.7513316
INFO:root:DEV Acc:		0.6399896(0.64)		0.958168(0.96)		0.8934299(0.89)
INFO:root:DEV L2:		0.5278211(0.53)		0.0674111(0.07)		0.1813849(0.18)
INFO:root:Epoch	15/50


INFO:root:DEV Goal Pricerange Acc: 0.9341843
INFO:root:DEV Goal Area Acc: 0.8751123
INFO:root:DEV Goal Name Acc: 0.9065421
INFO:root:DEV Goal Food Acc: 0.7513316
INFO:root:DEV Acc:		0.6415537(0.64)		0.9567256(0.96)		0.8934299(0.89)
INFO:root:DEV L2:		0.5256244(0.53)		0.0678134(0.07)		0.1813849(0.18)
INFO:root:Epoch	16/50


INFO:root:DEV Goal Pricerange Acc: 0.9297966
INFO:root:DEV Goal Area Acc: 0.8751123
INFO:root:DEV Goal Name Acc: 0.9065421
INFO:root:DEV Goal Food Acc: 0.7513316
INFO:root:DEV Acc:		0.6381648(0.64)		0.9560043(0.96)		0.8934299(0.89)
INFO:root:DEV L2:		0.5282167(0.53)		0.0690107(0.07)		0.1813849(0.18)
INFO:root:Epoch	17/50


INFO:root:DEV Goal Pricerange Acc: 0.9278022
INFO:root:DEV Goal Area Acc: 0.8751123
INFO:root:DEV Goal Name Acc: 0.9065421
INFO:root:DEV Goal Food Acc: 0.7513316
INFO:root:DEV Acc:		0.6373827(0.64)		0.9599712(0.96)		0.8934299(0.89)
INFO:root:DEV L2:		0.5296041(0.53)		0.0692763(0.07)		0.1813849(0.18)
INFO:root:Epoch	18/50


INFO:root:DEV Goal Pricerange Acc: 0.9114479
INFO:root:DEV Goal Area Acc: 0.8751123
INFO:root:DEV Goal Name Acc: 0.9065421
INFO:root:DEV Goal Food Acc: 0.7513316
INFO:root:DEV Acc:		0.6326903(0.63)		0.9556437(0.96)		0.8934299(0.89)
INFO:root:DEV L2:		0.5370054(0.54)		0.0716621(0.07)		0.1813849(0.18)


### Load Goal Pricerange Model

In [16]:
model_GoalPricerange = iDSTModel(vocabulary_size = VOCABULARY_SIZE,
                                 embedding_dim = EMBEDDING_DIM,
                                 altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                 hidden_dim = HIDDEN_DIM,
                                 output_dim = GOAL_PRICERANGE_DIM,
                                 device = DEVICE).to(DEVICE)
model_GoalPricerange.load_state_dict(torch.load("model_PANFMR_incremental_GoalPricerange.pt"))
model_GoalPricerange.eval()

iDSTModel(
  (embeddings): Embedding(1137, 170)
  (altered_embeddings): Linear(in_features=171, out_features=300, bias=True)
  (lstm): LSTM(300, 100)
  (classifier): Linear(in_features=100, out_features=5, bias=True)
)

### Load Goal Area Model

In [17]:
model_GoalArea = iDSTModel(vocabulary_size = VOCABULARY_SIZE,
                           embedding_dim = EMBEDDING_DIM,
                           altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                           hidden_dim = HIDDEN_DIM,
                           output_dim = GOAL_AREA_DIM,
                           device = DEVICE).to(DEVICE)
model_GoalArea.load_state_dict(torch.load("model_PANFMR_incremental_GoalArea.pt"))
model_GoalArea.eval()

iDSTModel(
  (embeddings): Embedding(1137, 170)
  (altered_embeddings): Linear(in_features=171, out_features=300, bias=True)
  (lstm): LSTM(300, 100)
  (classifier): Linear(in_features=100, out_features=7, bias=True)
)

### Load Goal Name Model

In [18]:
model_GoalName = iDSTModel(vocabulary_size = VOCABULARY_SIZE,
                           embedding_dim = EMBEDDING_DIM,
                           altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                           hidden_dim = HIDDEN_DIM,
                           output_dim = GOAL_NAME_DIM,
                           device = DEVICE).to(DEVICE)
model_GoalName.load_state_dict(torch.load("model_PANFMR_incremental_GoalName.pt"))
model_GoalName.eval()

iDSTModel(
  (embeddings): Embedding(1137, 170)
  (altered_embeddings): Linear(in_features=171, out_features=300, bias=True)
  (lstm): LSTM(300, 100)
  (classifier): Linear(in_features=100, out_features=115, bias=True)
)

### Load Goal Food Model

In [19]:
model_GoalFood = iDSTModel(vocabulary_size = VOCABULARY_SIZE,
                           embedding_dim = EMBEDDING_DIM,
                           altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                           hidden_dim = HIDDEN_DIM,
                           output_dim = GOAL_FOOD_DIM,
                           device = DEVICE).to(DEVICE)
model_GoalFood.load_state_dict(torch.load("model_PANFMR_incremental_GoalFood.pt"))
model_GoalFood.eval()

iDSTModel(
  (embeddings): Embedding(1137, 170)
  (altered_embeddings): Linear(in_features=171, out_features=300, bias=True)
  (lstm): LSTM(300, 100)
  (classifier): Linear(in_features=100, out_features=93, bias=True)
)

### Load Requested Model

In [20]:
model_Requested = iDSTModel(vocabulary_size = VOCABULARY_SIZE,
                            embedding_dim = EMBEDDING_DIM,
                            altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                            hidden_dim = HIDDEN_DIM,
                            output_dim = REQUESTED_DIM,
                            device = DEVICE,
                            is_requested = True).to(DEVICE)
model_Requested.load_state_dict(torch.load("model_PANFMR_incremental_Requested.pt"))
model_Requested.eval()

iDSTModel(
  (embeddings): Embedding(1137, 170)
  (altered_embeddings): Linear(in_features=171, out_features=300, bias=True)
  (lstm): LSTM(300, 100)
  (classifier): Linear(in_features=100, out_features=8, bias=True)
)

### Load Method Model

In [21]:
model_Method = iDSTModel(vocabulary_size = VOCABULARY_SIZE,
                         embedding_dim = EMBEDDING_DIM,
                         altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                         hidden_dim = HIDDEN_DIM,
                         output_dim = METHOD_DIM,
                         device = DEVICE).to(DEVICE)
model_Method.load_state_dict(torch.load("model_PANFMR_incremental_Method.pt"))
model_Method.eval()

iDSTModel(
  (embeddings): Embedding(1137, 170)
  (altered_embeddings): Linear(in_features=171, out_features=300, bias=True)
  (lstm): LSTM(300, 100)
  (classifier): Linear(in_features=100, out_features=5, bias=True)
)

### Print scores

In [30]:
dev_tracker = make_tracker(model_GoalPricerange,
                           model_GoalArea,
                           model_GoalName,
                           model_GoalFood,
                           model_Requested,
                           model_Method,
                           raw_X_dev, raw_Y_dev, dataset = "dstc2_dev", percentage = 1.0)
get_scores(dev_tracker, dataset = "dstc2_dev")


  File "dstc2/dstc2_scripts/score.py", line 238
    print(("state_component, stat, schedule, label_scheme, N, result"), file=csvfile)
                                                                            ^
SyntaxError: invalid syntax


{'goal_pricerange_accuracy': None,
 'goal_pricerange_l2': None,
 'goal_area_accuracy': None,
 'goal_area_l2': None,
 'goal_name_accuracy': None,
 'goal_name_l2': None,
 'goal_food_accuracy': None,
 'goal_food_l2': None,
 'goal_accuracy': None,
 'goal_l2': None,
 'requested_accuracy': None,
 'requested_l2': None,
 'method_accuracy': None,
 'method_l2': None}

In [23]:
test_tracker = make_tracker(model_GoalPricerange,
                            model_GoalArea,
                            model_GoalName,
                            model_GoalFood,
                            model_Requested,
                            model_Method,
                            raw_X_test, raw_Y_test, dataset = "dstc2_test", percentage = 1.0)
get_scores(test_tracker, dataset = "dstc2_test")

{'goal_pricerange_accuracy': 0.8944544,
 'goal_pricerange_l2': 0.1661498,
 'goal_area_accuracy': 0.8847761,
 'goal_area_l2': 0.188892,
 'goal_name_accuracy': 0.961326,
 'goal_name_l2': 0.0771567,
 'goal_food_accuracy': 0.7899989,
 'goal_food_l2': 0.3160805,
 'goal_accuracy': 0.6530086,
 'goal_l2': 0.5086604,
 'requested_accuracy': 0.9692655,
 'requested_l2': 0.0528308,
 'method_accuracy': 0.9238506,
 'method_l2': 0.1283662}